In [1]:
import os.path as osp
import os
import torch
from overflowDataset import OverFlowDataset
from torch_geometric.datasets import JODIEDataset
from tqdm import tqdm
from neighbor_sampler import NeighborSampler
import scipy
import numpy as np
from collections import OrderedDict
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected

# Set arguments
dataName = 'wiki' # 'overflow', 'taobao' , 'reddit', wiki'


In [2]:
# Load data
__file__ = os.path.abspath('')

print("Loading dataset ", dataName, "...")

if dataName == 'overflow':
    path = '/mnt/raid0nvme1/zz/data/' + 'overflow'
    dataset = OverFlowDataset(path)
    data = dataset[0]
    orig_edge_index = data.edge_index
elif dataName == 'taobao':
    path = '/mnt/raid0nvme1/zz/data/' + 'taobao/taobao.pt'
    data = torch.load(path)
    orig_edge_index = data.edge_index
    data.edge_index = to_undirected(data.edge_index)
elif dataName == 'reddit':
    path = '/mnt/raid0nvme1/zz/data/' + 'JODIE'
    dataset = JODIEDataset(path, name='reddit')
    data_orig = dataset[0]
    data = Data(x=data_orig.msg, edge_index=torch.stack([data_orig.src, data_orig.dst], dim=0), edge_attr=data_orig.t)
    orig_edge_index = data.edge_index
    data.edge_index = to_undirected(data.edge_index)
elif dataName == 'wiki':
    path = '/mnt/raid0nvme1/zz/data/' + 'JODIE'
    dataset = JODIEDataset(path, name='wikipedia')
    data_orig = dataset[0]
    data = Data(x=data_orig.msg, edge_index=torch.stack([data_orig.src, data_orig.dst], dim=0), edge_attr=data_orig.t)
    orig_edge_index = data.edge_index
    data.edge_index = to_undirected(data.edge_index)

print(data)

Loading dataset  wiki ...
Data(x=[157474, 172], edge_index=[2, 36514], edge_attr=[157474])


In [3]:
n1 = torch.unique(orig_edge_index[0])
n2 = torch.unique(orig_edge_index[1])
total_nodes = torch.unique(torch.cat((n1,n2))).numel()

loader = NeighborSampler(data.edge_index, sizes=[25,10], node_idx=torch.unique(torch.cat((n1,n2))), batch_size=1)

In [4]:
sample_cnt = {}
cnt = 0
pbar = tqdm(total=total_nodes)
for batch_size, ids, adjs in loader:
    sample_cnt[cnt] = len(ids)
    cnt +=1
    pbar.update(batch_size)
pbar.close()

100%|██████████| 9227/9227 [00:01<00:00, 7511.62it/s]


In [5]:
sample_cnt = {k: v for k, v in sorted(sample_cnt.items(), key=lambda item: item[1], reverse=True)}

In [20]:
ff = ' -1,10'
[int(size) for size in ff.split(',')]

[-1, 10]

In [6]:
list(sample_cnt.values())

[257,
 254,
 254,
 253,
 249,
 248,
 248,
 247,
 246,
 245,
 244,
 243,
 242,
 242,
 237,
 236,
 234,
 234,
 233,
 231,
 229,
 228,
 227,
 226,
 225,
 222,
 221,
 220,
 219,
 219,
 218,
 218,
 217,
 216,
 214,
 211,
 209,
 206,
 204,
 204,
 204,
 203,
 203,
 202,
 202,
 199,
 196,
 194,
 193,
 193,
 192,
 191,
 188,
 187,
 184,
 183,
 182,
 176,
 175,
 175,
 171,
 170,
 168,
 168,
 168,
 166,
 162,
 161,
 160,
 160,
 160,
 159,
 158,
 157,
 155,
 152,
 150,
 150,
 149,
 148,
 147,
 147,
 147,
 147,
 146,
 146,
 146,
 145,
 145,
 144,
 143,
 142,
 141,
 141,
 141,
 140,
 139,
 138,
 138,
 138,
 136,
 136,
 135,
 135,
 135,
 135,
 135,
 134,
 134,
 134,
 134,
 133,
 133,
 133,
 133,
 132,
 132,
 132,
 132,
 132,
 131,
 131,
 130,
 130,
 130,
 129,
 129,
 128,
 127,
 127,
 127,
 127,
 127,
 127,
 127,
 126,
 126,
 126,
 126,
 126,
 126,
 126,
 125,
 125,
 125,
 125,
 125,
 125,
 125,
 125,
 124,
 124,
 124,
 124,
 124,
 124,
 123,
 123,
 123,
 123,
 123,
 122,
 122,
 122,
 122,
 122,
 122

In [17]:
import json
path = '/mnt/raid0nvme1/zz/cache_data/'
with open(path + "wiki" + ".json", 'w') as fp:
    json.dump(sample_cnt, fp)